In [9]:
from NN_Thesis.nn_classes import *
from NN_Thesis.trainer import *
from NN_Thesis.helper_functions import *
from NN_Thesis.adapters import *
from NN_Thesis.helper_functions import *
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
from torch import optim


from torchvision.transforms import transforms
import numpy as np
import os
from PIL import Image
import random
import pandas as pd
from matplotlib import pyplot as plt

import BinaryNet.models as BNN_models
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

from NN_Thesis.dataset import cifar_n_dataset,get_split_dataset,cifar_100_split
from torch.utils.data import DataLoader



In [2]:
def normalize_channels(data):
    #We have a nxCxWxH array
    d = data
    d = torch.flatten(data,2,-1)
    mean= torch.mean(d,dim = [0,2])
    std = torch.std(d,dim = [0,2])
    return mean,std

In [3]:
train80_path = r'data\cifar-100\cifar-80\train'
test80_path =r'data\cifar-100\cifar-80\train'
cifar80_path = r'data\cifar-100\cifar-80'


train_80 = cifar_100_split(cifar80_path)
# train_80=apply_transforms(train_80)

test_80 = cifar_100_split(cifar80_path,train = False)
# test_80=apply_transforms(test_80,train = False)
mean,std,=normalize_channels(train_80.data)
mean,std

(tensor([0.5034, 0.4852, 0.4427]), tensor([0.2684, 0.2562, 0.2766]))

In [4]:
cifar20_path = r'data\cifar-100\cifar-20'


train_20 = cifar_100_split(cifar20_path)
test_20 = cifar_100_split(cifar20_path,train = False)


transform_train = transforms.Compose([  #resises the image so it can be perfect for our model.
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
                                      transforms.Normalize(mean,std) #Normalize all the images
                               ])





transform_test = transforms.Compose([  #resises the image so it can be perfect for our model.
                                      transforms.Normalize(mean,std) #Normalize all the images
                               ])

train_20.transform = transform_train
test_20.transform = transform_test
train20_DL = DataLoader(train_20,batch_size=64,shuffle= True,num_workers=4)
test20_DL = DataLoader(test_20,batch_size=64,shuffle= False,num_workers=4)

In [5]:
class placeholder(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    
    def forward(self,x):
        return x

In [21]:
def total_elem(net:nn.Module,per_layer = False,show=False):
    bin_total = 0
    for n,m in net.named_modules():
        if isinstance(m,(BinarizeConv2d,BinarizeLinear)):
            bin_total+= sum([p.numel() for p in m.parameters()])
    print(bin_total)
    total = 0
    for name,p in net.named_parameters():
        if per_layer:
            print(name,p.numel())
        total += p.numel()
    print(total)
    print(total - bin_total)
    return total,bin_total,total-bin_total

In [28]:
11_000_000*32/8000

44000.0

In [39]:

def weight_pct(weights,classes = 5):
    x,y,z = total_elem(resnet18_adapt(classes))

    base = (y+z*32)/8000

    return weights*32/8000/base

weight_pct(5105)



4330165
4336415
6250


0.036060496692725326

# Finetune

In [6]:
path = r'SavedModels\UniAdapt_Cifar100_Base\ResNet18_BNN_Baseline_2023-05-09 13-19-37\ResNet18_BNN_Baseline_best_acc.pth'

state = torch.load(path,map_location='cuda:0')

In [8]:
BNN = BNN_Resnet_UniAdapt(80)
BNN.load_state_dict(state,strict=False)
BNN.freeze()
BNN.cuda()

trainer = Trainer(model = BNN,seed = 123,model_name = f'Head Only',project_name = 'FINAL_CIFAR100_Regular',classes = train_20.classes,binarise= True)
m = trainer.model
m.to(trainer.device)
m.fc = BinarizeLinear(320,20)
m.bn3 = nn.BatchNorm1d(20)
# for layer in [m.fc,m.bn3]:
#     for p in layer.parameters():
#         p.requires_grad = True
#     layer.train()
trainer.lr = 1e-3
trainer.batch_size = 64
trainer.epochs =75
trainer.epoch_chkpts = []
trainer.start_epoch = 0
trainer.set_scheduler(None)
# adpt = getattr(m,after_layer)[-1]
# print(adpt)
# print(m.fc.weight[0,0])
# params = nn.ModuleList([m.fc,m.bn3,adpt]).parameters()
trainer.set_optimizer(torch.optim.Adam)
trainer.train(train20_DL,test20_DL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]}]}


Current Best Acc,▁▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁█▇▆▆▆▆▇▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇███████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7495
Current Best Acc,0.7495
Total Time (hours),0.55309
epoch,75


start
Project Name: FINAL_CIFAR100_Regular, Run Name Head Only 


Run Start : 2023-05-12 12-21-22
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4380700
Initial accuracy:
Test Accuracy : 4.2%, Test Loss: 34.74617124666833
best_acc.pth saved!
Test Accuracy : 4.7%, Test Loss: 34.512225687503815
best_acc.pth saved!
epoch: 1 average loss: 2.019
Test Accuracy : 59.0%, Test Loss: 1.5337960049510002
best_acc.pth saved!
Epoch Time (Training + Test) = 7.84 seconds
epoch: 2 average loss: 1.563
Test Accuracy : 62.6%, Test Loss: 1.4103223346173763
best_acc.pth 

In [7]:
BNN = BNN_Resnet_UniAdapt(80)
BNN.load_state_dict(state,strict=False)
BNN.cuda()

trainer = Trainer(model = BNN,seed = 123,model_name = f'Regular_Finetune',project_name = 'FINAL_CIFAR100_Regular',classes = train_20.classes,binarise= True)
m = trainer.model
m.to(trainer.device)
m.fc = BinarizeLinear(320,20)
m.bn3 = nn.BatchNorm1d(20)
# for layer in [m.fc,m.bn3]:
#     for p in layer.parameters():
#         p.requires_grad = True
#     layer.train()
trainer.lr = 1e-3
trainer.batch_size = 64
trainer.epochs =75
trainer.epoch_chkpts = []
trainer.start_epoch = 0
trainer.set_scheduler(None)
# adpt = getattr(m,after_layer)[-1]
# print(adpt)
# print(m.fc.weight[0,0])
# params = nn.ModuleList([m.fc,m.bn3,adpt]).parameters()
trainer.set_optimizer(torch.optim.Adam)
trainer.train(train20_DL,test20_DL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]}]}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny_suu. Use `wandb login --relogin` to force relogin


start
Project Name: FINAL_CIFAR100_Regular, Run Name Regular_Finetune 


Run Start : 2023-05-12 12-04-32
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4380700
Initial accuracy:
Test Accuracy : 6.2%, Test Loss: 30.395600713742006
best_acc.pth saved!
Test Accuracy : 6.2%, Test Loss: 30.302964508533478
epoch: 1 average loss: 1.993
Test Accuracy : 57.1%, Test Loss: 1.5776673816144466
best_acc.pth saved!
Epoch Time (Training + Test) = 14.16 seconds
epoch: 2 average loss: 1.547
Test Accuracy : 62.2%, Test Loss: 1.4067317582666874
best_acc.pth saved!
Epoc

# UniAdapter

In [7]:
path = r'SavedModels\UniAdapt_Cifar100_Base\ResNet18_BNN_Baseline_2023-05-09 13-19-37\ResNet18_BNN_Baseline_best_acc.pth'

state = torch.load(path,map_location='cuda:0')

## Pruning


In [14]:
for prune_amount in [0,0.8,0.9,0.95,0.99]:
    
    BNN = BNN_Resnet_UniAdapt(80,freeze_pre_weights=False)
    BNN.layer3[-1].do_bntan = True
    BNN.load_state_dict(state,strict=False)
    BNN.freeze()

    BNN = BNN.to('cuda:0')
    BNN.encoder_net = placeholder()

    n1 = uniAdapt_Net([80,160,320],[80,160,320],block = thinBlock3)
    n1 = n1.cuda()
    n1.train()

    adapt_net = nn.Sequential(n1,nn.Flatten())
    
    adapt_net.to('cuda:0')

    BNN.head_bn =  nn.BatchNorm1d(20)
    BNN.head =  BinarizeLinear(320 + 320,20)
    BNN.add_UniAdapter(adapt_net)
    BNN = BNN.to('cuda:0')
    BNN.uniAdapt = True
    trainer = Trainer(BNN,model_name=f'Cifar80_Prune_{prune_amount*100:.1f}%',project_name='FINAL_PRUNING_CIFAR100',binarise=True,classes=train_20.classes)
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [prune_amount,prune_amount]
    # trainer.pruning_rounds = 1 # Only need this for iterative pruning
    trainer.model_to_prune = trainer.model.uniAdaptNet[0]
    params = nn.ModuleList([BNN.uniAdaptNet,BNN.head_bn,BNN.head]).parameters()
    # params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
    print(trainer.model)
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    trainer.train(train20_DL,test20_DL,prune_strat='oneshot')

BNN_Resnet_UniAdapt(
  (conv1): BinarizeConv2d(3, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh1): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
  (tanh2): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (tanh1): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
      (conv2): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (tanh2): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
      (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(

Current Best Acc,▁▇█████
epoch,▁▂▃▅▆▇█
epoch time (s),▁▆▇▇█▇▆
lr,▁▁▁▁▁▁▁
test_accuracy,▁▇█████
test_loss,█▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁
Current Best Acc,0.72
epoch,6
epoch time (s),9.13606
lr,0.001


start
Project Name: FINAL_PRUNING_CIFAR100, Run Name Cifar80_Prune_0.0% 


Run Start : 2023-05-10 10-40-10
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4547660
Initial accuracy:
Test Accuracy : 4.9%, Test Loss: 41.67728718071227
best_acc.pth saved!
Test Accuracy : 4.5%, Test Loss: 41.409974575042725
epoch: 1 average loss: 1.618
Test Accuracy : 67.7%, Test Loss: 1.310125943273306
best_acc.pth saved!
Epoch Time (Training + Test) = 9.41 seconds
epoch: 2 average loss: 1.356
Test Accuracy : 69.3%, Test Loss: 1.2049964014440775
best_acc.pth saved!
Epoc

Current Best Acc,▁▇▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▇██▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▆▆▆▆▆▆▆▆▅▆▆▆▆▆▆▅
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇▇█████████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7635
Current Best Acc,0.7635
Total Time (hours),0.37894
epoch,75


start
Project Name: FINAL_PRUNING_CIFAR100, Run Name Cifar80_Prune_80.0% 


Run Start : 2023-05-10 10-51-47
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4547660
Initial accuracy:
Test Accuracy : 6.1%, Test Loss: 50.03438497652673
best_acc.pth saved!
Test Accuracy : 5.9%, Test Loss: 52.0525928735733
epoch: 1 average loss: 1.647
Test Accuracy : 66.9%, Test Loss: 1.3134911134839058
best_acc.pth saved!
Epoch Time (Training + Test) = 9.06 seconds
epoch: 2 average loss: 1.359
Test Accuracy : 69.1%, Test Loss: 1.1990348659455776
best_acc.pth saved!
Epoc

...

start
Project Name: FINAL_PRUNING_CIFAR100, Run Name Cifar80_Prune_90.0% 


Run Start : 2023-05-10 11-03-25
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4547660
Initial accuracy:
Test Accuracy : 4.9%, Test Loss: 46.913473578775005
best_acc.pth saved!
Test Accuracy : 5.5%, Test Loss: 46.263484835624695
best_acc.pth saved!
epoch: 1 average loss: 1.645
Test Accuracy : 66.6%, Test Loss: 1.3236247189342976
best_acc.pth saved!
Epoch Time (Training + Test) = 8.72 seconds
epoch: 2 average loss: 1.362
Test Accuracy : 69.0%, Test Loss: 1.2240473236888647
b

Current Best Acc,▁▇▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▇▇▇▇▇▇█▇▇▇█▇▆▇▆▇▇██▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇▇█████████████████▇▇▇███▇▇▇██▇████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7565
Current Best Acc,0.7565
Total Time (hours),0.36478
epoch,75


start
Project Name: FINAL_PRUNING_CIFAR100, Run Name Cifar80_Prune_95.0% 


Run Start : 2023-05-10 11-14-43
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4547660
Initial accuracy:
Test Accuracy : 3.5%, Test Loss: 52.68627118153177
best_acc.pth saved!
Test Accuracy : 3.2%, Test Loss: 52.512542366981506
epoch: 1 average loss: 1.653
Test Accuracy : 67.2%, Test Loss: 1.3136250413954258
best_acc.pth saved!
Epoch Time (Training + Test) = 8.82 seconds
epoch: 2 average loss: 1.350
Test Accuracy : 69.2%, Test Loss: 1.2027185447514057
best_acc.pth saved!
Ep

Current Best Acc,▁▇▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁█▇▇▇██▇▇▇████▇▇▇▇▇▇█▇▇▇█▇▇▇██████▇▇█▇▇▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇▇█████████████████▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇█
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.76
Current Best Acc,0.76
Total Time (hours),0.36455
epoch,75


start
Project Name: FINAL_PRUNING_CIFAR100, Run Name Cifar80_Prune_99.0% 


Run Start : 2023-05-10 11-26-00
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4547660
Initial accuracy:
Test Accuracy : 3.6%, Test Loss: 47.63883772929003
best_acc.pth saved!
Test Accuracy : 4.0%, Test Loss: 46.83586823940277
best_acc.pth saved!
epoch: 1 average loss: 1.635
Test Accuracy : 66.5%, Test Loss: 1.3140449598431587
best_acc.pth saved!
Epoch Time (Training + Test) = 8.79 seconds
epoch: 2 average loss: 1.351
Test Accuracy : 68.4%, Test Loss: 1.2102045435458422
bes

## Grouped Convolution

In [15]:

conv2_groups = [1,2,5,10,20,None]
conv1_groups = [None]*len(conv2_groups)

for conv1_group, conv2_group in zip(conv1_groups,conv2_groups):
    print(conv1_group,conv2_group)
    
    BNN = BNN_Resnet_UniAdapt(80,freeze_pre_weights=True)
    BNN.layer3[-1].do_bntan = True
    BNN.load_state_dict(state,strict=False)
    BNN.freeze()

    BNN = BNN.to('cuda:0')
    BNN.encoder_net = placeholder()

    n1 = uniAdapt_Net([80,160,320],[80,160,320],block = thinBlock2,conv1_group = conv1_group,conv2_group = conv2_group)
    n1 = n1.cuda()
    n1.train()

    adapt_net = nn.Sequential(n1,nn.Flatten())
    
    adapt_net.to('cuda:0')

    BNN.head_bn =  nn.BatchNorm1d(20)
    BNN.head =  BinarizeLinear(320 + 320,20)
    BNN.add_UniAdapter(adapt_net)
    BNN = BNN.to('cuda:0')
    BNN.uniAdapt = True
    print(BNN)
    trainer = Trainer(BNN,model_name=f'Grouped_Convolution_PW_Group_{conv2_group}',project_name='FINAL_CIFAR100_GROUPED_CONV',binarise=True,classes=train_20.classes)
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    # trainer.amount = [0.8,0]
    # trainer.pruning_rounds = 1 # Only need this for iterative pruning
    # trainer.model_to_prune = trainer.model.uniAdaptNet[0]
    params = nn.ModuleList([BNN.uniAdaptNet,BNN.head_bn,BNN.head]).parameters()
    # params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    trainer.train(train20_DL,test20_DL,prune_strat=None)

None 1
BNN_Resnet_UniAdapt(
  (conv1): BinarizeConv2d(3, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh1): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
  (tanh2): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (tanh1): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
      (conv2): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (tanh2): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
      (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), pa

Current Best Acc,▁▇▇▇████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▆▆▆▆▆▆▇▆▆▆▇▆▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆█▆▆▆▆▆▆▆▆▆▆
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇█▇████████████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7615
Current Best Acc,0.7615
Total Time (hours),0.36532
epoch,75


start
Project Name: FINAL_CIFAR100_GROUPED_CONV, Run Name Grouped_Convolution_PW_Group_1 


Run Start : 2023-05-10 11-40-32
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4547660
Initial accuracy:
Test Accuracy : 3.4%, Test Loss: 46.63080985682785
best_acc.pth saved!
Test Accuracy : 3.5%, Test Loss: 46.1002482175827
best_acc.pth saved!
epoch: 1 average loss: 1.647
Test Accuracy : 66.5%, Test Loss: 1.31552017852664
best_acc.pth saved!
Epoch Time (Training + Test) = 9.00 seconds
epoch: 2 average loss: 1.361
Test Accuracy : 68.2%, Test Loss: 1.2091489

Current Best Acc,▁▇▇▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇▇▇▇███████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.77
Current Best Acc,0.77
Total Time (hours),0.37634
epoch,75


start
Project Name: FINAL_CIFAR100_GROUPED_CONV, Run Name Grouped_Convolution_PW_Group_2 


Run Start : 2023-05-10 11-52-04
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4464460
Initial accuracy:
Test Accuracy : 2.6%, Test Loss: 52.408757616759864
best_acc.pth saved!
Test Accuracy : 2.1%, Test Loss: 52.519824266433716
epoch: 1 average loss: 1.687
Test Accuracy : 66.5%, Test Loss: 1.3659312166273594
best_acc.pth saved!
Epoch Time (Training + Test) = 17.44 seconds
epoch: 2 average loss: 1.384
Test Accuracy : 67.8%, Test Loss: 1.2432621195912361
best

...

start
Project Name: FINAL_CIFAR100_GROUPED_CONV, Run Name Grouped_Convolution_PW_Group_10 


Run Start : 2023-05-10 12-24-15
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4397900
Initial accuracy:
Test Accuracy : 4.4%, Test Loss: 52.47440204498874
best_acc.pth saved!
Test Accuracy : 4.3%, Test Loss: 50.28965163230896
epoch: 1 average loss: 1.792
Test Accuracy : 64.8%, Test Loss: 1.4008913673460484
best_acc.pth saved!
Epoch Time (Training + Test) = 8.77 seconds
epoch: 2 average loss: 1.456
Test Accuracy : 67.3%, Test Loss: 1.3142907433211803
best_a

Current Best Acc,▁▇▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁█▇▇▇▇▇▇▇▇▆▇▇▇▇▇▆▇▆█▇▇▆▇▆▇▇▇▇▇▆▇█▇▆▇█▇▆▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇██████████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7325
Current Best Acc,0.7325
Total Time (hours),0.35775
epoch,75


start
Project Name: FINAL_CIFAR100_GROUPED_CONV, Run Name Grouped_Convolution_PW_Group_20 


Run Start : 2023-05-10 12-35-18
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4389580
Initial accuracy:
Test Accuracy : 4.6%, Test Loss: 47.34286396974211
best_acc.pth saved!
Test Accuracy : 5.7%, Test Loss: 47.4720504283905
best_acc.pth saved!
epoch: 1 average loss: 1.813
Test Accuracy : 63.8%, Test Loss: 1.4363582953810692
best_acc.pth saved!
Epoch Time (Training + Test) = 8.59 seconds
epoch: 2 average loss: 1.463
Test Accuracy : 65.1%, Test Loss: 1.3291

Current Best Acc,▁▇▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▇█▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇█▇█▇▇█▇█▇▇█▇█▇▇█▇█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇██████████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.722
Current Best Acc,0.722
Total Time (hours),0.3574
epoch,75


start
Project Name: FINAL_CIFAR100_GROUPED_CONV, Run Name Grouped_Convolution_PW_Group_None 


Run Start : 2023-05-10 12-46-21
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4382060
Initial accuracy:
Test Accuracy : 5.4%, Test Loss: 46.566445198788
best_acc.pth saved!
Test Accuracy : 5.9%, Test Loss: 48.99974286556244
best_acc.pth saved!
epoch: 1 average loss: 1.838
Test Accuracy : 62.9%, Test Loss: 1.4534292481839657
best_acc.pth saved!
Epoch Time (Training + Test) = 8.53 seconds
epoch: 2 average loss: 1.481
Test Accuracy : 67.0%, Test Loss: 1.322

# Serial

## Seperate

## ALL 3

In [40]:
T,_,_ = total_elem(resnet18_adapt(20))
for group in [1,2,5,10,20]:

    conv_bn = conv_channel_adapter3(80,groups = group,kernel = 1, padding= 0,nonlinearity='relu')
    conv_bn2 = conv_channel_adapter3(160,groups = group,kernel = 1, padding= 0,nonlinearity='relu')
    conv_bn3 = conv_channel_adapter3(320,groups = group,kernel = 1, padding= 0,nonlinearity='relu')
    T,_,_ = total_elem(resnet18_adapt(80))
    total = 0
    for cnn in [conv_bn,conv_bn2,conv_bn3]:
        total += sum([p.numel() for p in cnn.parameters()])
    
    
    print(f'group {group} Total Elements {total} {weight_pct(total,20)*100:.0f}')


4334980
4341260
6280
4354240
4360640
6400
4334980
4341260
6280
group 1 Total Elements 136640 96
4354240
4360640
6400
4334980
4341260
6280
group 2 Total Elements 69440 49
4354240
4360640
6400
4334980
4341260
6280
group 5 Total Elements 29120 21
4354240
4360640
6400
4334980
4341260
6280
group 10 Total Elements 15680 11
4354240
4360640
6400
4334980
4341260
6280
group 20 Total Elements 8960 6


In [16]:


for group in [1,2,5,10,20]:

    conv_bn = conv_channel_adapter3(80,groups = group,kernel = 1, padding= 0,nonlinearity='relu')
    conv_bn2 = conv_channel_adapter3(160,groups = group,kernel = 1, padding= 0,nonlinearity='relu')
    conv_bn3 = conv_channel_adapter3(320,groups = group,kernel = 1, padding= 0,nonlinearity='relu')
    # adapt_fine2.add_adapter(after = 'bn2',adapter = bottleneck)
    
    # conv_bn = conv_channel_adapter3(input_channel,kernel = 1, padding= 0,groups= input_channel//10)
    adapt_fine2 = BNN_Resnet_UniAdapt(num_classes=80)
    
    adapt_fine2.load_state_dict(state,strict=False)
    
    # adapt_fine2.head = nn.Linear(320+160,80)
    # adapt_fine2.head_bn = nn.BatchNorm1d(480)
    adapt_fine2.freeze()
    # adapt_fine2.add_adapter(after = 'bn2',adapter = bottleneck)
    adapt_fine2.add_adapter(after = 'layer1',adapter = conv_bn)
    adapt_fine2.add_adapter(after = 'layer2',adapter = conv_bn2)
    adapt_fine2.add_adapter(after = 'layer3',adapter = conv_bn3)


    trainer = Trainer(model = adapt_fine2,seed = 123,model_name = f'3_adapter_groups_{group}',project_name = 'FINAL_CIFAR100_SERIAL_ALL3',classes = train_20.classes,binarise= True)
    m = trainer.model
    m.to(trainer.device)
    m.fc = BinarizeLinear(320,20)
    m.bn3 = nn.BatchNorm1d(20)
    # for layer in [m.fc,m.bn3]:
    #     for p in layer.parameters():
    #         p.requires_grad = True
    #     layer.train()
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.set_scheduler(None)
    # adpt = getattr(m,after_layer)[-1]
    # print(adpt)
    # print(m.fc.weight[0,0])
    # params = nn.ModuleList([m.fc,m.bn3,adpt]).parameters()
    trainer.set_optimizer(torch.optim.Adam)
    trainer.train(train20_DL,test20_DL)


No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}]}


Current Best Acc,▁▇██████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁██▇█▇▇██▇▇█▇▇█▇▇▇▇▇▇▇▇▇▇█▇█▇██▇▇█▇█▇▇█▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇█▇████████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7065
Current Best Acc,0.7065
Total Time (hours),0.35233
epoch,75


start
Project Name: FINAL_CIFAR100_SERIAL_ALL3, Run Name 3_adapter_groups_1 


Run Start : 2023-05-10 12-57-14
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4517340
Initial accuracy:
Test Accuracy : 5.8%, Test Loss: 32.381502734627695
best_acc.pth saved!
Test Accuracy : 5.2%, Test Loss: 31.557703912258148
epoch: 1 average loss: 1.769
Test Accuracy : 63.6%, Test Loss: 1.382547788321972
best_acc.pth saved!
Epoch Time (Training + Test) = 9.21 seconds
epoch: 2 average loss: 1.379
Test Accuracy : 68.3%, Test Loss: 1.2128282450139523
best_acc.pth saved!


Current Best Acc,▁▇██████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇█▇▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇██▇███████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7615
Current Best Acc,0.7615
Total Time (hours),0.38406
epoch,75


start
Project Name: FINAL_CIFAR100_SERIAL_ALL3, Run Name 3_adapter_groups_2 


Run Start : 2023-05-10 13-09-04
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4450140
Initial accuracy:
Test Accuracy : 4.2%, Test Loss: 32.15387351649582
best_acc.pth saved!
Test Accuracy : 5.2%, Test Loss: 32.55930799245834
best_acc.pth saved!
epoch: 1 average loss: 1.887
Test Accuracy : 62.4%, Test Loss: 1.4604874551296234
best_acc.pth saved!
Epoch Time (Training + Test) = 11.85 seconds
epoch: 2 average loss: 1.468
Test Accuracy : 65.6%, Test Loss: 1.2774135395884514


...

start
Project Name: FINAL_CIFAR100_SERIAL_ALL3, Run Name 3_adapter_groups_5 


Run Start : 2023-05-10 13-24-50
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4409820
Initial accuracy:
Test Accuracy : 5.1%, Test Loss: 39.75445556640625
best_acc.pth saved!
Test Accuracy : 5.5%, Test Loss: 38.93213331699371
best_acc.pth saved!
epoch: 1 average loss: 2.033
Test Accuracy : 57.8%, Test Loss: 1.5792807824909687
best_acc.pth saved!
Epoch Time (Training + Test) = 9.44 seconds
epoch: 2 average loss: 1.570
Test Accuracy : 61.7%, Test Loss: 1.409106869250536
be

Current Best Acc,▁▆▇▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁███▇▇███▇█████▇████▇▇▇██▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇▇▇███████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.74
Current Best Acc,0.74
Total Time (hours),0.3892
epoch,75


start
Project Name: FINAL_CIFAR100_SERIAL_ALL3, Run Name 3_adapter_groups_10 


Run Start : 2023-05-10 13-36-49
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4396380
Initial accuracy:
Test Accuracy : 4.9%, Test Loss: 31.124293260513596
best_acc.pth saved!
Test Accuracy : 5.8%, Test Loss: 32.35272818803787
best_acc.pth saved!
epoch: 1 average loss: 2.100
Test Accuracy : 57.0%, Test Loss: 1.6307434141635895
best_acc.pth saved!
Epoch Time (Training + Test) = 9.28 seconds
epoch: 2 average loss: 1.636
Test Accuracy : 60.7%, Test Loss: 1.4666546918451786

Current Best Acc,▁▆▇▇▇▇▇█████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁██▇████▇▇▇█████▇█▇▇▇██▇▇█▇██████▇▇█████
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇▇▇▇▇▇████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.726
Current Best Acc,0.726
Total Time (hours),0.38268
epoch,75


start
Project Name: FINAL_CIFAR100_SERIAL_ALL3, Run Name 3_adapter_groups_20 


Run Start : 2023-05-10 13-48-38
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : 123
model_architecture : <class 'NN_Thesis.nn_classes.BNN_Resnet_UniAdapt'>
binerised_training : True
Number of Elements : 4389660
Initial accuracy:
Test Accuracy : 4.2%, Test Loss: 34.03746459134825
best_acc.pth saved!
Test Accuracy : 4.2%, Test Loss: 33.77889347076416
epoch: 1 average loss: 2.166
Test Accuracy : 53.5%, Test Loss: 1.6771018952131271
best_acc.pth saved!
Epoch Time (Training + Test) = 9.14 seconds
epoch: 2 average loss: 1.676
Test Accuracy : 59.4%, Test Loss: 1.497622199356556
best_acc.pth saved!
E